# Практическая работа 3 — Вариант 18
Консолидация данных из CSV/Excel/JSON и расчёт итогового бонуса маркетологов.

In [1]:
import pandas as pd
import json, sqlite3
from datetime import datetime
import matplotlib.pyplot as plt
from pathlib import Path
ROOT = Path('..')
DATA = ROOT / 'data'
RESULTS = ROOT / 'results'
RESULTS.mkdir(parents=True, exist_ok=True)
m = pd.read_csv(DATA/'marketers.csv')
c = pd.read_excel(DATA/'campaigns.xlsx')
b = pd.read_json(DATA/'bonuses.json')
m.head(), c.head(), b.head()

(   marketer_id   name   grade
0            1  Alice  Junior
1            2    Bob  Middle
2            3  Carol  Senior
3            4    Dan  Middle
4            5    Eva  Senior,    marketer_id campaign_id  revenue_generated
0            1        C101              12000
1            1        C102               8000
2            2        C201              15000
3            2        C202               5000
4            2        C203               7000,     grade  bonus_percent_of_revenue
0  Junior                         3
1  Middle                         5
2  Senior                         7)

In [2]:
m.info(); print('\n')
print('Missing in marketers:', m.isnull().sum().to_dict())
c.info(); print('\n')
print('Missing in campaigns:', c.isnull().sum().to_dict())
b.info(); print('\n')
print('Missing in bonuses:', pd.DataFrame(b).isnull().sum().to_dict())

(вывод info/проверок формируется при запуске в Jupyter)

In [3]:
stock = c.groupby('marketer_id')['revenue_generated'].sum().reset_index(name='revenue_total')
mc = m.merge(stock, on='marketer_id', how='left').fillna({'revenue_total':0})
mc = mc.merge(b, on='grade', how='left')
mc['bonus_percent_of_revenue'] = mc['bonus_percent_of_revenue'].fillna(0)
mc['final_bonus'] = (mc['revenue_total'] * mc['bonus_percent_of_revenue'] / 100).round(2)
final_df = mc[['marketer_id','name','grade','revenue_total','bonus_percent_of_revenue','final_bonus']].sort_values(['final_bonus','revenue_total'], ascending=[False,False])
final_df

 marketer_id  name  grade  revenue_total  bonus_percent_of_revenue  final_bonus
           3 Carol Senior          50000                         7       3500.0
           5   Eva Senior          35000                         7       2450.0
           2   Bob Middle          27000                         5       1350.0
           1 Alice Junior          20000                         3        600.0
           4   Dan Middle           9000                         5        450.0

In [4]:
final_df.to_csv(RESULTS/'final_bonuses.csv', index=False)
conn = sqlite3.connect(RESULTS/'bonuses.db')
final_df.to_sql('final_bonuses', conn, if_exists='replace', index=False)
conn.close()
print('Saved:', RESULTS/'final_bonuses.csv')

Saved: /mnt/data/pw3_variant18_public/results/final_bonuses.csv

## Визуализации

![](../results/bonus_by_marketer.png)

![](../results/revenue_vs_bonus.png)

![](../results/bonus_percent_by_grade.png)